# Vulnerability Assessment (DSPy + Gemini)

This notebook loads a combined scanner report (e.g., Trivy + Grype), runs a DSPy/Gemini-based reasoning step to determine exploitability in-context, and saves JSON + Markdown reports.

## Prerequisites
- Python 3.10+
- In `vul_analysis/.env`, set `GEMINI_API_KEY` and optionally `GEMINI_MODEL` (default: `gemini-1.5-flash`).
- Install deps from this folder: `pip install -r requirements.txt`.

In [ ]:
# Optional: quick dependency check (safe to re-run)
import importlib, sys, subprocess, os
req_pkgs = ["pandas", "dotenv", "google.generativeai", "dspy"]
missing = [p for p in req_pkgs if importlib.util.find_spec(p) is None]
if missing:
    print("Installing missing packages:", missing)
    _req_path = os.path.join(os.path.dirname(__file__), 'requirements.txt') if '__file__' in globals() else 'requirements.txt'
    if os.path.exists(_req_path):
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-r', _req_path])
    else:
        for p in missing:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', p])
else:
    print("All required packages are available.")

In [ ]:
# Imports and paths
import os, json
from vul_analysis.run_assessment import assess

# This notebook resides in vul_analysis/. The scanner file is in ../Scanner/combined_results/
INPUT_FILE = r'..\Scanner\combined_results\hyperledger_fabric-peer_1.1.0_combined.json'
print('Input file exists:', os.path.exists(INPUT_FILE))
INPUT_FILE

In [ ]:
# Run the assessment
result = assess(INPUT_FILE)
result['summary_df'].head()

In [ ]:
# Output artifacts
print('Saved JSON:', result['json_path'])
print('Saved Markdown:', result['md_path'])
# Show a compact JSON preview
with open(result['json_path'], 'r', encoding='utf-8') as f:
    preview = json.load(f)
list(preview.keys())

## Notes
- If DSPy is unavailable or cannot be configured with Gemini, the pipeline automatically falls back to direct Gemini API calls.
- Outputs are written to `vul_analysis/outputs/` with timestamps.